In [130]:
import pandas as pd

In [131]:
census_df = pd.read_csv('github_data/census_data_cbg_extracted.csv')

In [132]:
census_df.dtypes

Total Population           float64
Median household income    float64
Median Gross Rent          float64
Median Age                 float64
state                       object
county                      object
tract                       object
block group                 object
Percent White              float64
dtype: object

In [133]:
# create unique id
census_df['CBG_ID'] = (census_df['state'].astype(str) + 
                       census_df['county'].astype(str).str.zfill(3) +
                       census_df['tract'].astype(str).str.zfill(6) + 
                       census_df['block group'].astype(str))
census_df['TRACT_ID'] = (census_df['state'].astype(str) + 
                       census_df['county'].astype(str).str.zfill(3) +
                       census_df['tract'].astype(str).str.zfill(6))
census_df.head()

,Total Population,Median household income,Median Gross Rent,Median Age,state,county,tract,block group,Percent White,CBG_ID,TRACT_ID
0,1016.0,NaN,432.0,30.8,36,005,006900,3,0.039370,360050069003,36005006900
1,712.0,67072.0,1593.0,42.1,36,005,041800,1,0.060393,360050418001,36005041800
2,978.0,74722.0,1628.0,37.5,36,005,016200,2,0.525562,360050162002,36005016200
3,2399.0,13783.0,435.0,37.2,36,005,020501,2,0.071697,360050205012,36005020501
4,668.0,20139.0,1108.0,26.9,36,005,017902,2,0.079341,360050179022,36005017902


In [134]:
# Drop if 0 population
mask = (census_df[['Total Population']] != 0).all(axis=1)
print(len(census_df) - sum(mask))
pre_filtered_df = census_df.loc[mask]
print(len(census_df) - len(pre_filtered_df))
pre_filtered_df = pre_filtered_df.dropna(subset=['Total Population'])
print(len(census_df) - len(pre_filtered_df))

271
271
275


In [135]:
nan_rows_count = pre_filtered_df.isna().any(axis=1).sum()
print(nan_rows_count)
nan_columns_count = pre_filtered_df.isna().any(axis=0).sum()
print(nan_columns_count)
nan_columns = pre_filtered_df.columns[pre_filtered_df.isna().any()].tolist()
print(f"Columns with any NaN values: {nan_columns}")

777
3
Columns with any NaN values: ['Median household income', 'Median Gross Rent', 'Median Age']


In [136]:
print(len(census_df))

6497


### Performing mean/median imputation for missing values

In [137]:
tract_df = pd.read_csv('github_data/census_data_tract_extracted.csv')

In [138]:
tract_df['TRACT_ID'] = (tract_df['state'].astype(str) + 
                       tract_df['county'].astype(str) +
                       tract_df['tract'].astype(str))

# Drop if 0 population
mask_t = (tract_df[['Total Population']] != 0).all(axis=1)
print(len(tract_df) - sum(mask_t))
pre_filtered_tract_df = tract_df.loc[mask_t]
print(len(tract_df) - len(pre_filtered_tract_df))
pre_filtered_tract_df = pre_filtered_tract_df.dropna(subset=['Total Population'])
print(len(tract_df) - len(pre_filtered_tract_df))


nan_rows_count = pre_filtered_tract_df.isna().any(axis=1).sum()
print(nan_rows_count)
nan_columns_count = pre_filtered_tract_df.isna().any(axis=0).sum()
print(nan_columns_count)
nan_columns = pre_filtered_tract_df.columns[pre_filtered_tract_df.isna().any()].tolist()
print(f"Columns with any NaN values: {nan_columns}")

pre_filtered_tract_df.head()

44
44
48
35
3
Columns with any NaN values: ['Median household income', 'Median Gross Rent', 'Median Age']


,Total Population,Median household income,Median Gross Rent,Median Age,state,county,tract,Percent White,TRACT_ID
0,1971.0,17778.0,494.0,39.5,36,005,015900,0.179604,36005015900
1,4812.0,31225.0,1374.0,30.4,36,005,017701,0.124480,36005017701
2,5786.0,33426.0,1137.0,32.2,36,005,017702,0.114760,36005017702
3,5006.0,37127.0,1219.0,29.9,36,005,017901,0.123452,36005017901
4,3701.0,23276.0,1182.0,32.6,36,005,017902,0.131046,36005017902


In [139]:
pre_filtered_tract_df.rename(columns={'Median Age': 'Imputed Median Age', 'Median Gross Rent': 'Imputed Median Gross Rent', 'Median household income': 'Imputed Median household income'}, inplace=True)
pre_filtered_tract_df = pre_filtered_tract_df[['TRACT_ID','Imputed Median Age','Imputed Median Gross Rent','Imputed Median household income']]

In [140]:
# pre_filtered_df = pd.merge(pre_filtered_df, pre_filtered_tract_df, on='TRACT_ID', how='left')
# pre_filtered_df['Median Age'] = pre_filtered_df['Median Age'].fillna(pre_filtered_df['Imputed Median Age'])
# pre_filtered_df.drop(columns=['Imputed Median Age','Imputed Median Gross Rent','Imputed Median household income'], inplace=True)

In [141]:
# Calculate the median 'Median Age' for each 'TRACT_ID' group
median_age_by_tract = pre_filtered_df.groupby('TRACT_ID')['Median Age'].transform('median')
# median_age_by_tract = pre_filtered_tract_df.groupby('TRACT_ID')['Median Age'].transform('median')

# Fill NaN values in 'Median Age' with the group-specific median value
pre_filtered_df['Median Age'] = pre_filtered_df['Median Age'].fillna(median_age_by_tract)

In [142]:
median_gr_by_tract = pre_filtered_df.groupby('TRACT_ID')['Median Gross Rent'].transform('median')
# median_gr_by_tract = pre_filtered_tract_df.groupby('TRACT_ID')['Median Gross Rent'].transform('median')

# Fill NaN values in 'Median Age' with the group-specific median value
pre_filtered_df['Median Gross Rent'] = pre_filtered_df['Median Gross Rent'].fillna(median_gr_by_tract)

median_HHincome_by_tract = pre_filtered_df.groupby('TRACT_ID')['Median household income'].transform('median')
# median_HHincome_by_tract = pre_filtered_tract_df.groupby('TRACT_ID')['Median household income'].transform('median')

# Fill NaN values in 'Median Age' with the group-specific median value
pre_filtered_df['Median household income'] = pre_filtered_df['Median household income'].fillna(median_HHincome_by_tract)

In [143]:
nan_rows_count = pre_filtered_df.isna().any(axis=1).sum()
print(nan_rows_count)
nan_columns_count = pre_filtered_df.isna().any(axis=0).sum()
print(nan_columns_count)
nan_columns = pre_filtered_df.columns[pre_filtered_df.isna().any()].tolist()
print(f"Columns with any NaN values: {nan_columns}")

57
3
Columns with any NaN values: ['Median household income', 'Median Gross Rent', 'Median Age']


In [147]:
pre_filtered_df.to_csv('github_data/census_for_merge.csv', index=False)

In [144]:
print(len(pre_filtered_df))

6222
